## Training ALLaM using LoRA + Tag, 15EPOCH

In [ ]:
import os, sys, json, time, random, numpy as np, torch, transformers
from pathlib import Path
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainerCallback
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers import set_seed

# -----------------------------
# ENV + Reproducibility
# -----------------------------
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
SEED = 42
set_seed(SEED)
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
torch.cuda.empty_cache()
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

print("="*60)
print("ENVIRONMENT CHECK")
print(f"Python executable: {sys.executable}")
print(f"Python version   : {sys.version}")
print(f"Torch version    : {torch.__version__}")
print(f"Transformers ver : {transformers.__version__}")
print(f"CUDA available   : {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"  Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA device detected!")
try:
    import datasets; print(f"Datasets version : {datasets.__version__}")
except Exception: print("Datasets not installed?")
try:
    import peft; print(f"PEFT version     : {peft.__version__}")
except Exception: print("PEFT not installed?")
print("="*60)

# -----------------------------
# PATHS / MODEL
# -----------------------------
TRAIN_PATH = Path("data_splits/train.jsonl")
DEV_PATH   = Path("data_splits/dev.jsonl")
OUTPUT_DIR = Path("outputs/allam7b-lora-token-15EPOCH")
BASE_MODEL = "ALLaM-AI/ALLaM-7B-Instruct-preview"
MAX_SEQ_LEN = 2048 

# -----------------------------
# LoRA config
# -----------------------------
lora_config = LoraConfig(
    r=32,  
    lora_alpha=64,  
    lora_dropout=0.1,  
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

# -----------------------------
# Training args (TRL SFTConfig; TRL==0.9.6)
# ----------------------------

train_args = SFTConfig(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=15,  
    per_device_train_batch_size=2, 
    gradient_accumulation_steps=8,  
    learning_rate=5e-5,  
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.1,  
    bf16=True,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    gradient_checkpointing=True,
    report_to="tensorboard",
    save_total_limit=3,  # Keep more checkpoints
    seed=SEED,
    packing=True,
    dataset_text_field="text",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    # Add these for better logging
    logging_dir=str(OUTPUT_DIR / "logs"),
    logging_first_step=True,
    remove_unused_columns=False,
)
# -----------------------------
# Data IO
# -----------------------------
def load_jsonl(path: Path) -> Dataset:
    rows = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip(): continue
            obj = json.loads(line)
            instr = (obj.get("instruction") or "").strip()
            resp  = (obj.get("response") or "").strip()
            if instr and resp:
                rows.append({"instruction": instr, "response": resp})
    if not rows:
        raise ValueError(f"No valid rows found in {path}")
    return Dataset.from_list(rows)

train_ds = load_jsonl(TRAIN_PATH)
dev_ds   = load_jsonl(DEV_PATH)

EOS = "</s>"
def fmt(example):
    instr = example["instruction"] 
    resp  = example["response"]
    text = f"### Instruction:\n{instr}\n\n### Response:\n{resp}{EOS}"
    return {"text": text}

train_text = train_ds.map(fmt, remove_columns=[c for c in train_ds.column_names if c!="text"])
dev_text   = dev_ds.map(fmt,   remove_columns=[c for c in dev_ds.column_names if c!="text"])

print(f"Train examples: {len(train_text):,} | Dev examples: {len(dev_text):,}")

# -----------------------------
# Tokenizer & Base model
# -----------------------------
tok = AutoTokenizer.from_pretrained(BASE_MODEL)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.config.use_cache = False

# -----------------------------
# Step logging callback
# -----------------------------
class LoggingCallback(TrainerCallback):
    def __init__(self):
        self.t0 = time.time()
        self.tlast = self.t0

    def on_step_end(self, args, state, control, logs=None, **kwargs):
        if not state.is_local_process_zero:
            return
        logs = logs or {}
        now = time.time()
        train_loss = logs.get("loss")
        if train_loss is not None:
            print(
                f"[Step {state.global_step}] epoch={state.epoch:.2f} "
                f"train_loss={train_loss:.4f} "
                f"lr={logs.get('learning_rate', 'N/A'):.2e} "
                f"dt={now-self.tlast:.2f}s elapsed={(now-self.t0)/60:.2f}m",
                flush=True
            )
            self.tlast = now

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if not state.is_local_process_zero:
            return
        metrics = metrics or {}
        now = time.time()
        
        eval_loss = metrics.get("eval_loss")
        if eval_loss is not None:
            print(
                f"[EVAL] epoch={state.epoch:.2f} "
                f"eval_loss={eval_loss:.4f} "
                f"dt={now-self.tlast:.2f}s elapsed={(now-self.t0)/60:.2f}m",
                flush=True
            )
            self.tlast = now

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not state.is_local_process_zero:
            return
        logs = logs or {}
        now = time.time()
        
        # Log any metrics that weren't caught above
        if "loss" in logs or "eval_loss" in logs:
            train_loss = logs.get("loss", "N/A")
            eval_loss = logs.get("eval_loss", "N/A")
            lr = logs.get("learning_rate", "N/A")
            
            print(
                f"[LOG] epoch={state.epoch:.2f} "
                f"train_loss={train_loss} "
                f"eval_loss={eval_loss} "
                f"lr={lr} "
                f"dt={now-self.tlast:.2f}s elapsed={(now-self.t0)/60:.2f}m",
                flush=True
            )
            self.tlast = now

# -----------------------------
# Trainer (TRL + LoRA)
# -----------------------------
trainer = SFTTrainer(
    model=model,
    tokenizer=tok,  
    train_dataset=train_text,
    eval_dataset=dev_text,
    peft_config=lora_config,
    args=train_args,
    max_seq_length=MAX_SEQ_LEN,  
)


try:
    trainer.model.print_trainable_parameters()
except Exception:
    pass

trainer.add_callback(LoggingCallback())

print("Starting training…", flush=True)
trainer.train()

# -----------------------------
# Save adapters & tokenizer
# -----------------------------
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
trainer.save_model()
tok.save_pretrained(OUTPUT_DIR)
print("✅ Done. LoRA adapters saved to:", OUTPUT_DIR)

# -----------------------------
# Final dev evaluation + perplexity
# -----------------------------
metrics = trainer.evaluate()
print("Final evaluation metrics:", metrics)
if "eval_loss" in metrics and metrics["eval_loss"] is not None:
    try:
        ppl = float(np.exp(metrics["eval_loss"]))
        print(f"Perplexity: {ppl:.3f}")
    except Exception:
        pass

# -----------------------------
# Save run config snapshot
# -----------------------------
with open(OUTPUT_DIR / "run_config.txt", "w", encoding="utf-8") as f:
    f.write(f"Seed: {SEED}\n")
    f.write(f"Base model: {BASE_MODEL}\n")
    f.write(f"Train path: {TRAIN_PATH}\n")
    f.write(f"Dev path  : {DEV_PATH}\n")
    f.write(f"Epochs: {train_args.num_train_epochs}\n")
    f.write(f"LR: {train_args.learning_rate}\n")
    f.write(f"Per-device batch: {train_args.per_device_train_batch_size}\n")
    f.write(f"Grad accum: {train_args.gradient_accumulation_steps}\n")
    f.write(f"Max seq len: {MAX_SEQ_LEN}\n")
    f.write(f"BF16: {train_args.bf16}\n")
    f.write(f"LORA r/alpha/drop: {lora_config.r}/{lora_config.lora_alpha}/{lora_config.lora_dropout}\n")
    f.write(f"Transformers: {transformers.__version__}\n")
    f.write(f"Torch: {torch.__version__}\n")
print("Saved run_config.txt")


ENVIRONMENT CHECK
Python executable: /usr/bin/python
Python version   : 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Torch version    : 2.2.1+cu121
Transformers ver : 4.42.3
CUDA available   : True
CUDA device count: 1
  Device 0: NVIDIA RTX 6000 Ada Generation
Datasets version : 2.19.1
PEFT version     : 0.11.1


Map:   0%|          | 0/5122 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

Train examples: 5,122 | Dev examples: 588


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


[LOG] epoch=13.69 train_loss=1.3529 eval_loss=N/A lr=1.006426501190233e-07 dt=131.02s elapsed=44.31m
[LOG] epoch=13.98 train_loss=N/A eval_loss=1.9792847633361816 lr=N/A dt=63.19s elapsed=45.36m
[EVAL] epoch=13.98 eval_loss=1.9793 dt=0.00s elapsed=45.36m


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


[LOG] epoch=14.05 train_loss=N/A eval_loss=1.9791737794876099 lr=N/A dt=20.92s elapsed=45.71m
[EVAL] epoch=14.05 eval_loss=1.9792 dt=0.00s elapsed=45.71m
✅ Done. LoRA adapters saved to: outputs/allam7b-lora-token-15EPOCH


[LOG] epoch=14.05 train_loss=N/A eval_loss=1.892530918121338 lr=N/A dt=9.95s elapsed=45.87m
[EVAL] epoch=14.05 eval_loss=1.8925 dt=0.00s elapsed=45.87m
Final evaluation metrics: {'eval_loss': 1.892530918121338, 'eval_runtime': 6.8253, 'eval_samples_per_second': 3.663, 'eval_steps_per_second': 0.586, 'epoch': 14.054054054054054}
Perplexity: 6.636
Saved run_config.txt


In [2]:
print("Best eval_loss:", trainer.state.best_metric)
print("Best checkpoint path:", trainer.state.best_model_checkpoint)


Best eval_loss: 1.892530918121338
Best checkpoint path: outputs/allam7b-lora-token-15EPOCH/checkpoint-97


## Training ALLaM using LoRA + No Tag, 15EPOCH 

In [ ]:
    # -----------------------------
    # PATHS / MODEL
    # -----------------------------
    TRAIN_PATH = Path("data_splits/train.jsonl")
    DEV_PATH   = Path("data_splits/dev.jsonl")
    OUTPUT_DIR = Path("outputs/allam7b-lora-no-token-15EPOCH")
    BASE_MODEL = "ALLaM-AI/ALLaM-7B-Instruct-preview"
    MAX_SEQ_LEN = 2048

    # -----------------------------
    # LoRA config
    # -----------------------------
    lora_config = LoraConfig(
        r=32,  
        lora_alpha=64,  
        lora_dropout=0.1,  
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
    )

    # -----------------------------
    # Training args (TRL SFTConfig; TRL==0.9.6)
    # -----------------------------

    train_args = SFTConfig(
        output_dir=str(OUTPUT_DIR),
        num_train_epochs=15,  
        per_device_train_batch_size=2,  
        gradient_accumulation_steps=8,  
        learning_rate=5e-5, 
        lr_scheduler_type="cosine_with_restarts",
        warmup_ratio=0.1,  
        bf16=True,
        logging_steps=10,
        save_strategy="epoch",
        eval_strategy="epoch",
        gradient_checkpointing=True,
        report_to="tensorboard",
        save_total_limit=3,  
        seed=SEED,
        packing=True,
        dataset_text_field="text",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        logging_dir=str(OUTPUT_DIR / "logs"),
        logging_first_step=True,
        remove_unused_columns=False,
    )
    # -----------------------------
    # Data IO
    # -----------------------------
    def load_jsonl(path: Path) -> Dataset:
        rows = []
        with path.open("r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                obj = json.loads(line)
                instr = (obj.get("instruction") or "").strip()
                resp  = (obj.get("response") or "").strip()
                if instr and resp:
                    rows.append({"instruction": instr, "response": resp})
        if not rows:
            raise ValueError(f"No valid rows found in {path}")
        return Dataset.from_list(rows)

    train_ds = load_jsonl(TRAIN_PATH)
    dev_ds   = load_jsonl(DEV_PATH)
    TAG_RE = re.compile(r'^\s*<\s*DIALECT\s*=\s*(HIJAZI|NAJDI)\s*>\s*', re.IGNORECASE)
    EOS = "</s>"

    removed_ct = 0
    total_ct   = 0
    def fmt_no_tag(example):
        global removed_ct, total_ct
        instr = example["instruction"]
        total_ct += 1
        stripped = TAG_RE.sub("", instr).lstrip()
        if stripped != instr:
            removed_ct += 1
        resp = example["response"]
        text = f"### Instruction:\n{stripped}\n\n### Response:\n{resp}{EOS}"
        return {"text": text}

    train_text = train_ds.map(fmt_no_tag, remove_columns=[c for c in train_ds.column_names if c!="text"])
    dev_text   = dev_ds.map(fmt_no_tag,   remove_columns=[c for c in dev_ds.column_names if c!="text"])

    print(f"Train examples: {len(train_text):,} | Dev examples: {len(dev_text):,}")
    print(f"Leading <DIALECT=…> tags removed in formatting: {removed_ct}/{total_ct} "
        f"({(removed_ct/max(1,total_ct))*100:.2f}%)")

    # -----------------------------
    # Tokenizer & Base model
    # -----------------------------
    tok = AutoTokenizer.from_pretrained(BASE_MODEL)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    model.config.use_cache = False

    # -----------------------------
    # Step logging
    # -----------------------------
    class LoggingCallback(TrainerCallback):
        def __init__(self):
            self.t0 = time.time()
            self.tlast = self.t0

        def on_step_end(self, args, state, control, logs=None, **kwargs):
            """Log every training step"""
            if not state.is_local_process_zero:
                return
            logs = logs or {}
            now = time.time()
            train_loss = logs.get("loss")
            if train_loss is not None:
                print(
                    f"[Step {state.global_step}] epoch={state.epoch:.2f} "
                    f"train_loss={train_loss:.4f} "
                    f"lr={logs.get('learning_rate', 'N/A'):.2e} "
                    f"dt={now-self.tlast:.2f}s elapsed={(now-self.t0)/60:.2f}m",
                    flush=True
                )
                self.tlast = now

        def on_evaluate(self, args, state, control, metrics=None, **kwargs):
            """Log evaluation results"""
            if not state.is_local_process_zero:
                return
            metrics = metrics or {}
            now = time.time()
            
            eval_loss = metrics.get("eval_loss")
            if eval_loss is not None:
                print(
                    f"[EVAL] epoch={state.epoch:.2f} "
                    f"eval_loss={eval_loss:.4f} "
                    f"dt={now-self.tlast:.2f}s elapsed={(now-self.t0)/60:.2f}m",
                    flush=True
                )
                self.tlast = now

        def on_log(self, args, state, control, logs=None, **kwargs):
            """Fallback logging"""
            if not state.is_local_process_zero:
                return
            logs = logs or {}
            now = time.time()
            if "loss" in logs or "eval_loss" in logs:
                train_loss = logs.get("loss", "N/A")
                eval_loss = logs.get("eval_loss", "N/A")
                lr = logs.get("learning_rate", "N/A")
                
                print(
                    f"[LOG] epoch={state.epoch:.2f} "
                    f"train_loss={train_loss} "
                    f"eval_loss={eval_loss} "
                    f"lr={lr} "
                    f"dt={now-self.tlast:.2f}s elapsed={(now-self.t0)/60:.2f}m",
                    flush=True
                )
                self.tlast = now



    # -----------------------------
    # Trainer (TRL + LoRA)
    # -----------------------------
    trainer = SFTTrainer(
        model=model,
        tokenizer=tok,  
        train_dataset=train_text,
        eval_dataset=dev_text,
        peft_config=lora_config,
        args=train_args,
        max_seq_length=MAX_SEQ_LEN,  
    )


    try:
        trainer.model.print_trainable_parameters()
    except Exception:
        pass

    trainer.add_callback(LoggingCallback())
    print("Starting NO-TAG training…", flush=True)
    trainer.train()

    # -----------------------------
    # Save adapters & tokenizer
    # -----------------------------
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    trainer.save_model()
    tok.save_pretrained(OUTPUT_DIR)
    print("✅ Done. LoRA (NO TAG) adapters saved to:", OUTPUT_DIR)

    # -----------------------------
    # Final dev evaluation + perplexity
    # -----------------------------
    metrics = trainer.evaluate()
    print("Final evaluation metrics:", metrics)
    if "eval_loss" in metrics and metrics["eval_loss"] is not None:
        try:
            ppl = float(np.exp(metrics["eval_loss"]))
            print(f"Perplexity: {ppl:.3f}")
        except Exception:
            pass

    # -----------------------------
    # Save run config snapshot
    # -----------------------------
    with open(OUTPUT_DIR / "run_config.txt", "w", encoding="utf-8") as f:
        f.write(f"Seed: {SEED}\n")
        f.write(f"Base model: {BASE_MODEL}\n")
        f.write(f"Train path: {TRAIN_PATH}\n")
        f.write(f"Dev path  : {DEV_PATH}\n")
        f.write(f"Epochs: {train_args.num_train_epochs}\n")
        f.write(f"LR: {train_args.learning_rate}\n")
        f.write(f"Per-device batch: {train_args.per_device_train_batch_size}\n")
        f.write(f"Grad accum: {train_args.gradient_accumulation_steps}\n")
        f.write(f"Max seq len: {MAX_SEQ_LEN}\n")
        f.write(f"BF16: {train_args.bf16}\n")
        f.write(f"LORA r/alpha/drop: {lora_config.r}/{lora_config.lora_alpha}/{lora_config.lora_dropout}\n")
        f.write(f"Transformers: {transformers.__version__}\n")
        f.write(f"Torch: {torch.__version__}\n")
    print("Saved run_config.txt")


NO-TAG TRAIN — ENVIRONMENT CHECK
Python executable: /usr/bin/python
Python version   : 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Torch version    : 2.2.1+cu121
Transformers ver : 4.42.3
CUDA available   : True
CUDA device count: 1
  Device 0: NVIDIA RTX 6000 Ada Generation
Datasets version : 2.19.1
PEFT version     : 0.11.1


Map:   0%|          | 0/5122 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

Train examples: 5,122 | Dev examples: 588
Leading <DIALECT=…> tags removed in formatting: 5710/5710 (100.00%)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


[LOG] epoch=13.74 train_loss=1.5617 eval_loss=N/A lr=4.6861723431538276e-07 dt=124.48s elapsed=39.95m
[LOG] epoch=13.98 train_loss=N/A eval_loss=2.1778223514556885 lr=N/A dt=50.17s elapsed=40.78m
[EVAL] epoch=13.98 eval_loss=2.1778 dt=0.00s elapsed=40.78m


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


[LOG] epoch=14.55 train_loss=1.5522 eval_loss=N/A lr=0.0 dt=92.55s elapsed=42.32m
[LOG] epoch=14.55 train_loss=N/A eval_loss=2.177546501159668 lr=N/A dt=8.13s elapsed=42.46m
[EVAL] epoch=14.55 eval_loss=2.1775 dt=0.00s elapsed=42.46m
✅ Done. LoRA (NO TAG) adapters saved to: outputs/allam7b-lora-no-token-15EPOCH


[LOG] epoch=14.55 train_loss=N/A eval_loss=2.1006088256835938 lr=N/A dt=9.82s elapsed=42.62m
[EVAL] epoch=14.55 eval_loss=2.1006 dt=0.00s elapsed=42.62m
Final evaluation metrics: {'eval_loss': 2.1006088256835938, 'eval_runtime': 6.3647, 'eval_samples_per_second': 3.614, 'eval_steps_per_second': 0.471, 'epoch': 14.545454545454545}
Perplexity: 8.171
Saved run_config.txt


In [5]:
print("Best eval_loss:", trainer.state.best_metric)
print("Best checkpoint path:", trainer.state.best_model_checkpoint)


Best eval_loss: 2.1006088256835938
Best checkpoint path: outputs/allam7b-lora-no-token-15EPOCH/checkpoint-86
